# **CD2201-16: Ciencia de Datos en Astronomía**

**Profesores:** Boris Cuevas, Paula Díaz, Danilo Sepúlveda, Francisco Urbina

# **Tarea 4: Analizando un cubo de datos**

En clases ya vimos varios tipos de datos que utilizamos para analizar objetos astronómicos, como lo hacen las técnicas de fotometria y espectroscopia. En principio, se puede entender la fotometría como un "subconjunto" de la espectroscopia, pero esta última es mucho más costosa.

Con un único espectro solo podemos analizar cómo se descompone la luz de un único objeto, lo cual puede ser muy útil si estudiamos objetos puntuales. En algunos casos queremos estudiar objetos "extendidos" en el cielo, como las galaxias o explorar los espectros de muchas estrellas de forma simultánea (como en un cúmulo de estrellas). Para este tipo de casos, es ideal obtener un espectro para cada posición del cielo alrededor de algún objeto central. Esto se suele conocer como "Espectroscopia de campo integral" (IFS por sus siglas en inglés).

En esta minitarea exploraremos un cubo de datos, aprenderemos a como entender su formato, extraer información fundamental e identificar algunos objetos extrayendo su espectro.

## A jugar con los datos!

Descargue los datos que vamos a utilizar en el siguiente [link.](https://drive.google.com/file/d/1V7j-WJpTY9fZwajyhpp_Vj9ZZ1rjKN_j/view?usp=sharing)

**CUIDADO:** los datos pesan ~3 Gb, usted debe descargarlos y subirlos a alguna carpeta de Drive (misma carpeta donde tendrá esta tarea!).

In [ ]:
# Partiremos por "montar" el drive, esto permite que podamos leer los archivos de nuestro drive
from google.colab import drive
drive.mount('/content/drive')    # se les debe abrir una pestaña extra pidiendo permisos


In [ ]:
# Encuentren la ruta de su archivo, para ello puede utilizar la barra de herramientas izquierda
# lleguen a su archivo desde su "drive" y con el click derecho "copiar ruta" y luego la pegan
# como un string en la siguiente variable.
path_archivo = ''

Cargue el cubo de datos con astropy en la siguiente celda.

In [ ]:
import astropy.io.fits as F
cubo =

Muestre el *header* de la primera extensión de la variable cubo.

In [ ]:
# Muestre el header (es recomendable que no use "print" en esta ocasión)

Examinando la información desplegada responda:

1.   ¿Qué telescopio tomó los datos?
2.   ¿Qué instrumento lo hizo?
3.   ¿Cuál es el nombre del objeto observado?
4.   Reporte el valor de la variable 'EXPTIME' del header. Esto corresponde al *tiempo de integración* que se necesito para obtener los datos. ¿A qué corresponde ese tiempo?

**Respuesta:**

Ahora que ya conoce el telescopio y el instrumento que tomó los datos, investigue y responda las siguientes preguntas:

1. ¿De que tamaño es el diámetro del espejo del telescopio?
2. ¿Cuán grande es el campo de visión que examina el instrumento (Field of view o FOV en inglés)? Compare dicho campo de visión con el tamaño angular de la luna visto desde la tierra (algo así como medio grado) ¿Cuántos campos de visión del intrumento "caben" en la luna?.
3. ¿Qué rango de longitud de onda explora este instrumento?

**Respuesta:**

Explore la extensión asociada a los datos (índice = 1).
Extraiga los datos y describa cada eje del cubo. ¿En que unidades se encuentran los datos?
**Respuesta:**

**Hint:** explore las variables 'CTYPE', estas le indicarán a que corresponde cada eje del cubo. Las unidades se encontrarán en la variable 'BUNIT'.

In [ ]:
# Muestre el header de la primera extensión (índice = 1) aquí.

In [ ]:
# Guarde los datos en la siguiente variable.
datos =

In [ ]:
print(datos.shape)     # noten que la dimensión de mayor número es la espectral, las otras 2 son espaciales :)

## White-light image y su visualización

Una white-light image es una imagén en la cual se sumó en la dirección espectral, es decir, sumamos todos los fotones que llegaron en todo el rango explorado, de ahí el origen del nombre.

Cree un código capaz de calcular una white-light image (una línea con numpy), quizá se demore un poco en ejecutar esta parte.

In [ ]:
# Aquí el código.
import numpy as np
white_light_image =

Visualice su imagen con plt.imshow().

In [ ]:
import matplotlib.pyplot as plt

Notará que en la imagen existen una especie de arcos aproximadamente circulares. Explique brevemente, ¿a qué fenómeno físico corresponde esto?

**Respuesta:**

El resultado crudo de dicha visualización puede mejorar de forma significativa.
Para ello usted debe:
1. Cortar la imagen para enforcarnos en las figuras "semi circulares" que se aprecian.
2. Ajustar la scala de la imagen para generar mayor contraste.
3. Cambie el mapa de colores a uno que le parezca adecuado ([aquí una lista](https://matplotlib.org/stable/users/explain/colors/colormaps.html)).
4. Añadir coordenadas a los ejes y una colorbar.


In [ ]:
# Solución
from astropy.wcs import WCS
# Utilice el siguiente wcs para añadir las coordenadas a su visualización.
wcs = WCS(cubo[1].header)
wcs = wcs[0,:,:]

# Haga su gráfico a continuación:

Para lo que viene utilizaremos una librería creada para lidiar con cubos de datos.
La instalaremos utilizando **pip**, note que esta instalación es solo en el ambiente de google colab, nada se instalará en su computador.  
La documentación la puede encontrar [aquí.](https://mpdaf.readthedocs.io/en/stable/)

In [ ]:
!pip install mpdaf

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import mpdaf as mpdaf

# Aquí tienen una lista de 4 coordenadas extraídas de la Tabla 1 del artículo https://arxiv.org/pdf/2203.13759
# s0(a) =  9:21:25.040, 18:10:12.27    (RA, DEC)
# s0(b) =  9:21:25.858, 18:10:07.24    (RA, DEC)
# s0(c) =  9:21:26.439, 18:10:13.60    (RA, DEC)
# s0(d) =  9:21:25.781, 18:10:24.64    (RA, DEC)

# Cree un objeto SkyCoord para cada una de las coordenadas anteriores.
s0a =
s0b =
s0c =
s0d =

# Esto mostrará las coordenadas en grados de la primera coordenada.
print('s0a = (RA, DEC) = ',s0a.ra.value,',' ,s0a.dec.value)

Cargaremos el cubo con mpdaf y luego extraeremos espectros con aperturas circulares.

In [ ]:
# Cargamos con mpdaf
cube = mpdaf.obj.Cube(path_archivo)

Busque la documentación de mpdaf.obj.Cube, allí encontrará el método "aperture".
Este método extrae un espectro de una apertura circular en el cubo.

Averigue como utilizar dicha función para extraer un espectro en cada coordenada antes creada
considerando, una apertura de diámetro de 2"
Un pequeño hint: el centro debe ser de la forma (dec, ra) y cada coordenada con su valor en grados.

In [ ]:
# Rellene el código a continuación reemplazando el "CENTRO" y "radio", según corresponda.
# Note que ya le hemos indicado al método las unidades de medida, por lo que debe ser consistente.

espectro_s0a = cube.aperture(CENTRO, RADIO, unit_center=u.deg, unit_radius=u.arcsec)
espectro_s0b = cube.aperture(CENTRO, RADIO, unit_center=u.deg, unit_radius=u.arcsec)
espectro_s0c = cube.aperture(CENTRO, RADIO, unit_center=u.deg, unit_radius=u.arcsec)
espectro_s0d = cube.aperture(CENTRO, RADIO, unit_center=u.deg, unit_radius=u.arcsec)

Grafique los 4 espectros en un único gráfico. Notará que es bastante difícil descifrar algo de forma directa con plots "crudos". Estos espectros se ven todos bastante parecidos.

Grafique únicamente el rango de longitud de onda entre 9240-9310 Å.
Para esto puede utilizar el método "plot" de los objetos antes creados.

In [ ]:
# Inserte su gráfico a continuación:


Notará que hay una emisión bastante clara con 2 máximos, uno detrás de otro.
Esto es lo que se conoce como líneas espectrales de emisión de una galaxia.
En particular, estas líneas parecen ser todas iguales entre los distintos espectros. Los autores del artículo antes mencionado afirman que estas 4 coordenadas provienen de una misma única fuente de fondo, ¿cómo justificarían dicha afirmación con el gráfico que obtuvieron recién?

**Respuesta: (algo corto y simple!)**

Esta emisión proviene del gas de estas fuentes, en particular es una línea de  OII, es decir, oxígeno que fue ionizado. Esta línea es lo que se conoce como un "doblete" (doublet en inglés), lo que quiere decir que en realidad son dos líneas, muy cercanas en longitud de onda. La longitud de onda medida en laboratorio de ambas líneas es 3727.092 Å y 3729.875 Å, respectivamente. Claramente las líneas que vemos en estas fuentes no están centradas en esas longitudes de onda.

El desplazamiento aparente de las longitudes de onda, con respecto a aquellas medidas en el laboratorio, es un efecto de la expansión del universo. La expansión genera que los fotones pasen a tener una longitud de onda cada vez más larga a medida que se alejan de un observador. Esto se suele definir con la cantidad "redshift" (z) definida por:

$$\lambda_{obs} = (1+z) \lambda_0 $$

donde $\lambda_{obs}$ y $\lambda_0$ corresponde a la longitud de onda observada y aquella medida en el laboratorio. Vamos a determinar el redshift de esta fuente, para ello fitearemos un modelo que considera dos gaussianas (una para cada línea) y una constante, es decir nuestro modelo es de la forma:

$$f(\lambda, z,  A_1, \sigma_1, A_2, \sigma_2, C) = A_1 \exp(-(\lambda - (1+z)\lambda_{0,1})^2 /(2 \sigma_1^2 ))  + A_2 \exp(-(\lambda - (1+z)\lambda_{0,2})^2 /(2 \sigma_2^2 )) + C$$

Donde $\lambda_{0,1} = 3727.092$ Å y $\lambda_{0,2} = 3729.875$ Å.

MPDAF ya es capaz de hacer este fit por nosotros, para ello proceda como se le indica:

1. Note que cada espectro antes producido es un objeto de Python, dichos objetos tienen como método gauss_dfit. Explore la documentación de mpdaf y busque el método gauss_dfit. Describa los 3 primeros parámetros de dicha función.
**Respuesta:**
2. Considerando que vamos a hacer nuestro fit en el rango 9240-9310 Å, utilice el método antes descrito.

3. El resultado deberían ser dos objetos de tipo obj.Gauss1D, utilice el método print_param() para mostrar los resultados de cada fit.

In [ ]:
# Utilice el método gauss_dfit (solo utilizaremos un espectro de los antes calculados, no necesitamos los 4)
fit1, fit2 = espectro_s0a.gauss_dfit()  # rellene con los parámetros adecuados

# Haga el print de los parámetros del primer objeto Gauss1D (fit1) (utilice el método "print_param" para esto)


In [ ]:
# Haga el print de los parámetros del segundo objeto Gauss1D (fit2)


Note que los modelos no reportan las variables $\sigma$ antes descritas, sino que para describir el ancho utilizan la variable fwhm lo cual es una abreviación para la palabra Full width at half Maximum (ancho a media altura en español).

Responda:
- ¿Qué es el FWHM?
- Para una distribución normal, escriba una ecuación que relacione el FWHM con la desviación estándar $\sigma$ (no es necesario que la demuestre).  

**Respuesta:**

Ahora, veremos que tan bien lo hizo este modelo, para ello graficaremos encima del espectro el modelo que se obtuvo.

In [ ]:
# Función de ayuda para la siguiente parte :)

def gaussian(x, center, fwhm, peak):
  sigma = fwhm/2.355
  dif = x - center
  return peak * np.exp(-(dif**2)/(2*sigma**2))

Realice un gráfico que contenga el espectro en el cual se realizó el "fit" y el modelo encontrado.
Para ello debe implementar la función $f(\lambda, z,  A_1, \sigma_1, A_2, \sigma_2, C)$. Esto se puede realizar utilizando la función "gaussian" del bloque anterior o puede hacer su propia implementación desde 0.

Solo grafique el rango entre 9240-9310 angstroms. Recuerde incluír alguna leyenda, nombres en los ejes, título, etc.

In [ ]:
# Inserte aquí su gráfico:

Finalmente, con el resultado del fit, podemos encontrar el redshift de la fuente. Para ello simplemente calcule el redshift (z) utilizando la definición provista, considerando que la longitud de onda observada corresponde con algún centro de las gaussianas.


In [ ]:
# Escriba aquí código para el cálculo:

lambda_0 =  3727.092  # consideramos la línea de longitud de onda más corta
z =
print('El redshift es: {:.3f}'.format(z))

## Opcional: no todo es Python!

Existe software capaz de proveer de una visualización y análisis rápido de datos como los que utilizamos en esta actividad. Uno de ellos especialmente diseñado para lidiar con cubos de datos es QfitsView.
El software se puede descargar en el siguiente [link](https://www.mpe.mpg.de/~ott/QFitsView/).

Descargue el software y abra directamente los datos descargados en su computador, para ello debe seleccionar la extensión "DATA". Notará que abrirá el cubo y podrá desplazarse a lo largo de todos los canales. Ajuste la escala a "linear" y utilizando el 98% del rango de los datos para un mejor contraste.

Una pequeña guía para repetir todo lo que hicieron en la actividad:
- A medida que mueve su "ratón" por sobre la imagen del canal seleccionado, verá que el "gráfico" de abajo cambia. Dicho gráfico por defecto es el espectro asociado al pixel en el cual su "ratón" está.
- Podemos fijar el espectro que queremos visualizar, para ello haga click derecho en alguna fuente que quiera observar con detalle y seleccione "lock position".
- Para mejorar la señal a ruido de los espectros es recomendable sumar la contribución de varios pixeles al mismo tiempo, para ello, en el gráfico del espectro en su parte superior, seleccione "Circular" en vez de "Single pixel" y aumente el radio (donde dice "R1") a unos 5 pixeles.  
- Si desea analizar en detalle dicho espectro, lo pueden guardar como un archivo fits, para ello solo deben apretar "click derecho" encima de la fuente de interés (sugiero una galaxia muy brillante en el sur-este cerca del arco gravitacional). También pueden guardarlo en otro "buffer" y solo cambiarse de buffer para ver cada espectro.

Pueden aquí ver una serie de [videos tutorial](https://www.youtube.com/playlist?list=PLdVESrjTNUXtwxwysIDLoC4pV20Lbvqaz) si desean aprender más.